# Generating a FlipBook from Random Burst Images

CS 543 Final Project


In [ ]:
!pip install opencv-python tqdm

In [2]:
import os
import random
import cv2
import shutil
import string
from tqdm import tqdm
import numpy as np
import time
import matplotlib.pyplot as plt

# Our algorithm files
import SIFT_Ransac_Naive

In [ ]:
input_data_directory = 'data'
temp_directory = "temp"
output_directory = "output"

for directory in [input_data_directory, temp_directory, output_directory]:
    if not os.path.exists(directory):
        os.makedirs(directory)

In [ ]:
files = os.listdir(input_data_directory)
video_files = [file for file in files if os.path.isfile(
    os.path.join(input_data_directory, file))]

video_filename = None
# video_filename = "filename" # choose specific filename
if video_files:
    video_filename = random.choice(video_files)
    print(f"Randomly selected file: {video_filename}")
else:
    raise ValueError("ERROR: No video files found in the directory.")

video_filename_shortened = os.path.splitext(video_filename)[0]

In [4]:
# Delete any pre-existing temp data
for dir_name in os.listdir(temp_directory):
    dir_path = os.path.join(temp_directory, dir_name)
    if os.path.isdir(dir_path):
        shutil.rmtree(dir_path)

In [ ]:
def extract_random_frames(video_file, percentage):
    cap = cv2.VideoCapture(os.path.join(input_data_directory, video_file))
    if not cap.isOpened():
        print("Error: Cannot open video file.")
        return

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    sample_size = max(1, int(percentage * total_frames))

    frame_indices = sorted(random.sample(range(total_frames), sample_size))

    output_folder = os.path.join(temp_directory, video_file)
    os.makedirs(output_folder, exist_ok=True)

    frame_count = 0
    current_index = 1
    saved_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count in frame_indices:
            output_path = os.path.join(
                output_folder, "{:05}.jpg".format(current_index))
            cv2.imwrite(output_path, frame)
            saved_count += 1
            print(f"Got Image {current_index} / {len(frame_indices)}: {100 * (current_index / float(len(frame_indices))): .3f}%", end="\r", flush=True)
            current_index += 1

        frame_count += 1
        if saved_count >= sample_size:
            break

    cap.release()
    print(f"Extracted {saved_count} frames to '{output_folder}'.")
    return output_folder

sampling_percentage = 0.075
temp_data_folder = extract_random_frames(video_filename, sampling_percentage)

In [ ]:
def create_ground_truth_video_from_images(image_folder, output_video, framerate=30):
    images = sorted(
        [f for f in os.listdir(image_folder) if f.endswith(".jpg")],
        key=lambda x: int(os.path.splitext(x)[0])
    )

    if not images:
        print("No images found in the specified folder.")
        return

    first_image_path = os.path.join(image_folder, images[0])
    frame = cv2.imread(first_image_path)
    height, width, layers = frame.shape
    size = (width, height)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, framerate, size)

    for image in images:
        image_path = os.path.join(image_folder, image)
        frame = cv2.imread(image_path)
        out.write(frame)

    out.release()
    print(f"Ground Truth Video created successfully: {output_video}")

ground_truth_filepath = os.path.join(output_directory, f"{video_filename_shortened}_gt.MP4")
create_ground_truth_video_from_images(temp_data_folder, ground_truth_filepath)

In [ ]:
def generate_random_name(length=10):
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))

def rename_files_in_directory(directory):
    files = os.listdir(directory)
    original_num_files = len(files)
    
    jpg_files = [f for f in files if f.lower().endswith('.jpg')]
    
    used_names = set()

    for file in jpg_files:
        while True:
            new_name = generate_random_name()
            if new_name not in used_names:
                used_names.add(new_name)
                break
        
        original_path = os.path.join(directory, file)
        new_path = os.path.join(directory, f"{new_name}.jpg")
        
        os.rename(original_path, new_path)
        # print(f"Renamed {file} to {new_name}.jpg")
    
    files = os.listdir(directory)
    new_num_files = len(files)
    if new_num_files == original_num_files:
        print(f"Original Num Images == New Num Images in {directory}")
    else:
        raise ValueError(f"Original Num Images != New Num Images in {directory}")
    
rename_files_in_directory(temp_data_folder)

In [8]:
def compute_mse(video_path1, video_path2):
    cap1 = cv2.VideoCapture(video_path1)
    cap2 = cv2.VideoCapture(video_path2)

    if not cap1.isOpened() or not cap2.isOpened():
        print("Error opening video files.")
        return None

    fps1 = cap1.get(cv2.CAP_PROP_FPS)
    fps2 = cap2.get(cv2.CAP_PROP_FPS)
    frame_count1 = int(cap1.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_count2 = int(cap2.get(cv2.CAP_PROP_FRAME_COUNT))

    if fps1 != fps2:
        print("Error: Videos have different frame rates.")
        return None

    num_frames = min(frame_count1, frame_count2)

    total_error = 0

    for i in range(num_frames):
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()

        if not ret1 or not ret2:
            print("Error reading frames.")
            break

        if frame1.shape != frame2.shape:
            frame2 = cv2.resize(frame2, (frame1.shape[1], frame1.shape[0]))

        error = (frame1.astype(np.float32) - frame2.astype(np.float32)) ** 2
        frame_mse = np.mean(error)

        total_error += frame_mse

    mean_mse = total_error / num_frames

    cap1.release()
    cap2.release()

    return mean_mse

In [ ]:
algorithms = [SIFT_Ransac_Naive]
runtimes = {}
mses = {}

for algorithm in algorithms:
    print(f"Running {algorithm.__name__}")
    model_output_filepath = os.path.join(output_directory, f"{video_filename_shortened}_{algorithm.__name__}.mp4")
    start = time.now()
    algorithm.run(temp_data_folder, model_output_filepath, 30)
    duration = time.now() - start
    print(f"{algorithm.__name__} took {duration:.4f} seconds")
    mse = compute_mse(ground_truth_filepath, model_output_filepath)
    print(f"Mean Squared Error (MSE) between the two videos for {video_filename_shortened} and algorithm {algorithm.__name__}: {mse:.5f}")
    runtimes[algorithm.__name__] = duration
    mses[algorithm.__name__] = mse

In [ ]:
runtime_keys, runtime_values = zip(*runtimes.items())
mse_keys, mse_values = zip(*mses.items())

fig, axes = plt.subplots(1, 2, figsize=(12, 6))
fig.suptitle(f"Comparison of Runtimes and Mean Squared Errors (MSEs) of different algorithms for {video_filename_shortened}", fontsize=16, weight='bold')

def add_values_to_bars(ax, keys, values):
    for i, value in enumerate(values):
        ax.text(i, value + 0.02, f"{value:.2f}", ha='center', va='bottom', fontsize=10)

axes[0].bar(runtime_keys, runtime_values, color='skyblue')
axes[0].set_title("Runtimes")
axes[0].set_xlabel("Algorithm")
axes[0].set_ylabel("Duration (seconds)")
add_values_to_bars(axes[0], runtime_keys, runtime_values)

axes[1].bar(mse_keys, mse_values, color='lightgreen')
axes[1].set_title("Mean Squared Error")
axes[1].set_xlabel("Algorithm")
axes[1].set_ylabel("Mean Squared Error (MSE)")
add_values_to_bars(axes[1], mse_keys, mse_values)

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig(os.path.join(output_directory, f'{video_filename_shortened}_data.jpg'), dpi=300, bbox_inches='tight')
# plt.show()
plt.close()